In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors as NN

In [18]:
X = [[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]]
nbrs = NN(n_neighbors=3).fit(X)
distances, indices = nbrs.kneighbors([[0,-0.5]])
indices 

array([[0, 3, 1]])

In [19]:
distances

array([[ 1.11803399,  1.80277564,  2.06155281]])

In [20]:
dataretrieve = pd.read_csv("..//datasets/dataretrieve.csv", na_values='?')
desharnais = pd.read_csv("..//datasets/desharnais.csv", na_values='?')
dataretrieve = dataretrieve.fillna(0)
desharnais = desharnais.fillna(0)

In [21]:
with open("..//datasets/dataretrieve.txt") as infile:
    dataretrievelist = infile.read()
    dataretrievelist = dataretrievelist.split(',')
with open("..//datasets/desharnais.txt") as infile:
    desharnaislist = infile.read()
    desharnaislist = desharnaislist.split(',')

In [25]:
from sklearn.model_selection import KFold

In [26]:
#para compreensao
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis